In [95]:
# plot the intersection of two 3d cuboids

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
import scipy.spatial.transform.rotation as scipyR
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from mmdet3d.structures import BaseInstance3DBoxes, BboxOverlaps3D
import torch


In [24]:
import sys
sys.path.append('/home/cws-ml-lab/mmdetection3d_for_rail/sutherland-hodgman')

from SH import PolygonClipper

In [84]:
stadler_blue = (30, 90, 154)
stadler_yellow = (255, 189, 0)
stadler_grey = (192,192,192)

# devide by 255
stadler_blue = (30/255, 90/255, 154/255)
stadler_yellow = (255/255, 189/255, 0/255)
stadler_grey = (192/255, 192/255, 192/255)

In [15]:
stadler_blue = (30, 90, 154)
stadler_yellow = (255, 189, 0)
stadler_grey = (192,192,192)

# devide by 255
stadler_blue = (30/255, 90/255, 154/255)
stadler_yellow = (255/255, 189/255, 0/255)
stadler_grey = (192/255, 192/255, 192/255)

In [16]:
def plot_bbox(ax, bbox):
    """
    Function to plot a 3D bounding box.
    
    Args:
    - ax: Matplotlib 3D axis.
    - bbox: List or array of the bounding box in the format (x, y, z, w, l, h, yaw).
    """
    # Unpack the bounding box parameters
    x, y, z, w, l, h, yaw = bbox
    
    # Define the eight vertices of the bounding box in local coordinates
    vert = np.array([[w/2, l/2, h/2],
                     [-w/2, l/2, h/2],
                     [-w/2, -l/2, h/2],
                     [w/2, -l/2, h/2],
                     [w/2, l/2, -h/2],
                     [-w/2, l/2, -h/2],
                     [-w/2, -l/2, -h/2],
                     [w/2, -l/2, -h/2]])
    
    # Rotation matrix based on yaw (rotation around the z-axis)
    r = scipyR.Rotation.from_euler('xyz', [0.0, 0.0, yaw], degrees=False)
    Rot = r.as_matrix()

    # Rotate the vertices
    rotated_vert_norm = np.dot(Rot, vert.T).T

    # Translate the vertices to the actual position in world coordinates
    rotated_vert = rotated_vert_norm + np.array([x, y, z])

    # Define the 12 edges of the bounding box (pairs of vertices that form edges)
    edges = [[0, 1], [1, 2], [2, 3], [3, 0], # top face
             [4, 5], [5, 6], [6, 7], [7, 4], # bottom face
             [0, 4], [1, 5], [2, 6], [3, 7]] # side edges
    
    # Draw each edge of the bounding box
    for edge in edges:
        ax.plot3D(*zip(*rotated_vert[edge]), color="blue")

    # To enhance the visualization, you can also add a transparent face
    faces = [[0, 1, 2, 3], [4, 5, 6, 7], # top and bottom faces
             [0, 1, 5, 4], [2, 3, 7, 6], # side faces
             [0, 3, 7, 4], [1, 2, 6, 5]] # side faces
    
    # Create a 3D polygon collection for the faces
    ax.add_collection3d(Poly3DCollection([rotated_vert[face] for face in faces], 
                                         facecolors='cyan', linewidths=1, edgecolors='r', alpha=0.25))


In [167]:
def get_vertices(bbox):
    
    x, y, z, w, l, h, yaw = bbox
    
    # Define the eight vertices of the bounding box in local coordinates
    
    # Define the eight vertices of the bounding box in local coordinates
    vert = np.array([[w/2, l/2, h/2],
                     [w/2, -l/2, h/2],
                     [-w/2, -l/2, h/2],
                     [-w/2, l/2, h/2],
                     [w/2, l/2, -h/2],
                     [w/2, -l/2, -h/2],
                     [-w/2, -l/2, -h/2],
                     [-w/2, l/2, -h/2]])
    
    # Rotation matrix based on yaw (rotation around the z-axis)
    r = scipyR.Rotation.from_euler('xyz', [0.0, 0.0, yaw], degrees=False)
    Rot = r.as_matrix()

    # Rotate the vertices
    rotated_vert_norm = np.dot(Rot, vert.T).T

    # Translate the vertices to the actual position in world coordinates
    rotated_vert = rotated_vert_norm + np.array([x, y, z])
    
    return rotated_vert

def get_2d_vertices(bbox):
    
    rotated_vert = get_vertices(bbox)
    
    # only take top 4 vertices
    rotated_vert = rotated_vert[:4]
    
    # Project the vertices to the ground plane
    proj_vert = rotated_vert[:, [0, 1]]
    
    return proj_vert

def get_intersection_2d(bbox1, bbox2):
    
    bbox1_2d = get_2d_vertices(bbox1)
    bbox2_2d = get_2d_vertices(bbox2)
    
    print(bbox1_2d)
    print(bbox2_2d)
    
    clip = PolygonClipper(warn_if_empty = True)

    clipped_polygon = clip(bbox1_2d,bbox2_2d)
    
    return clipped_polygon

def get_intersection_3d(bbox1, bbox2):
    
    bbox1_2d = get_2d_vertices(bbox1)
    bbox2_2d = get_2d_vertices(bbox2)
    
    clip = PolygonClipper(warn_if_empty = True)
    
    clipped_polygon = clip(bbox1_2d,bbox2_2d)
    
    vertices_bbox1 = get_vertices(bbox1)
    vertices_bbox2 = get_vertices(bbox2)
    
    # h_max = max(vertices_bbox1[:, 2].max(), vertices_bbox2[:, 2].max())
    # h_min = min(vertices_bbox1[:, 2].min(), vertices_bbox2[:, 2].min())
    
    h_values = [vertices_bbox1[:, 2].max(), vertices_bbox1[:, 2].min(), vertices_bbox2[:, 2].max(), vertices_bbox2[:, 2].min()]
    
    # sort list ascending
    h_values.sort()
    
    h_min = h_values[1]
    h_max = h_values[2]
    
    vertices_top = np.hstack((clipped_polygon, h_max*np.ones((clipped_polygon.shape[0], 1))))
    vertices_bottom = np.hstack((clipped_polygon, h_min*np.ones((clipped_polygon.shape[0], 1))))
    
    return vertices_top, vertices_bottom

def plot_bbox3d(ax, vertices):
    
    rotated_vert = vertices
    
    # Define the 12 edges of the bounding box (pairs of vertices that form edges)
    edges = [[0, 1], [1, 2], [2, 3], [3, 0], # top face
                [4, 5], [5, 6], [6, 7], [7, 4], # bottom face
                [0, 4], [1, 5], [2, 6], [3, 7]] # side edges
    
    # Draw each edge of the bounding box
    for edge in edges:
        ax.plot3D(*zip(*rotated_vert[edge]), color=stadler_yellow)

    # To enhance the visualization, you can also add a transparent face
    faces = [[0, 1, 2, 3], [4, 5, 6, 7], # top and bottom faces
                [0, 1, 5, 4], [2, 3, 7, 6], # side faces
                [0, 3, 7, 4], [1, 2, 6, 5]] # side faces
    
    # Create a 3D polygon collection for the faces
    ax.add_collection3d(Poly3DCollection([rotated_vert[face] for face in faces], 
                                         facecolors=stadler_yellow, linewidths=1, alpha=0.125))
    
    
def plot_intersection_3d(ax, vertices):
    
    vertices_top, vertices_bottom = vertices
    
    # Define the 12 edges of the bounding box (pairs of vertices that form edges)
    # edges = [[0, 1], [1, 2], [2, 3], [3, 4], [4, 5], [5, 6], [6, 7], [7, 0],
    #          [8, 9], [9, 10], [10, 11], [11, 12], [12, 13], [13, 14], [14, 15], [15, 8],
    #          [0, 8], [1, 9], [2, 10], [3, 11], [4, 12], [5, 13], [6, 14], [7, 15]] # side edges
    
    for i in range(vertices_top.shape[0]):
        ax.plot3D(*zip(*vertices_top[[i, (i+1)%vertices_top.shape[0]]]), color=stadler_blue)
        ax.plot3D(*zip(*vertices_bottom[[i, (i+1)%vertices_bottom.shape[0]]]), color=stadler_blue)
        ax.plot3D(*zip(*np.vstack((vertices_top[i], vertices_bottom[i]))), color=stadler_blue)
        
    ax.add_collection3d(Poly3DCollection([vertices_top], facecolors=stadler_blue, linewidths=1, edgecolors=stadler_blue, alpha=0.25))
    ax.add_collection3d(Poly3DCollection([vertices_bottom], facecolors=stadler_blue, linewidths=1, edgecolors=stadler_blue, alpha=0.25))
    
    for i in range(vertices_top.shape[0]):
        ax.add_collection3d(Poly3DCollection([np.vstack((vertices_top[i], vertices_top[(i+1)%vertices_top.shape[0]], vertices_bottom[(i+1)%vertices_bottom.shape[0]], vertices_bottom[i]))], 
                                         facecolors=stadler_blue, linewidths=1, edgecolors=stadler_blue, alpha=0.25))
        
def add_plot(ax, bbox1, bbox2):
    
    bbox1_vertices = get_vertices(bbox1)
    bbox2_vertices = get_vertices(bbox2)
    
    plot_bbox3d(ax, bbox1_vertices)
    plot_bbox3d(ax, bbox2_vertices)
    
    intersection = get_intersection_3d(bbox1, bbox2)
    
    plot_intersection_3d(ax, intersection)
    
    base_bbox1 = BaseInstance3DBoxes(torch.tensor([bbox1]))
    base_bbox2 = BaseInstance3DBoxes(torch.tensor([bbox2]))
    
    iou = base_bbox1.overlaps(base_bbox1, base_bbox2)
    iou_rounded = round(iou.item(), 2)
    
    ax.set_title(f'IoU: {iou_rounded}')
    
    # ax.set_xlabel('X')
    # ax.set_ylabel('Y')
    # ax.set_zlabel('Z')
    
    # axis need to be proportional
    # ax.set_box_aspect((1, 1, 1))

    # # set axis limits
    ax.set_xlim(-1.5, 1.5)
    ax.set_ylim(-1.5, 1.5)
    ax.set_zlim(-1.5, 1.5)
    
    # no ticks
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_zticks([])
        

In [ ]:
# 5 subplots horizontal
fig = plt.figure(figsize=(15, 3))

bbox1 = [0.0, 0.0, 0.0, 2.0, 1.0, 1.0, 0.0]
bbox2 = [0.66, 0.0, 0.0, 2.0, 1.0, 1.0, 0.0]

ax1 = fig.add_subplot(151, projection='3d')
add_plot(ax1, bbox1, bbox2)

# add bbox1 and bbox2 to the plot as text under the plot
# bbox1_text = f'Size box 1: [{bbox1[3]}, {bbox1[4]},{bbox1[5]}]'
# bbox2_text = f'Size box 2: [{bbox2[3]}, {bbox2[4]},{bbox2[5]}]'
# ax1.text2D(0.5, -0.05, bbox1_text, transform=ax1.transAxes, ha='center', c='black')
# ax1.text2D(0.5, -0.12, bbox2_text, transform=ax1.transAxes, ha='center', c='black')

bbox1 = [0.0, 0.0, 0.0, 2.0, 1.0, 1.0, 0.0]
bbox2 = [0.66, 0.0, 0.33, 2.0, 1.0, 1.0, 0.0]

ax2 = fig.add_subplot(152, projection='3d')
add_plot(ax2, bbox1, bbox2)

# add bbox1 and bbox2 to the plot as text under the plot
# bbox1_text = f'Size box 1: [{bbox1[3]}, {bbox1[4]},{bbox1[5]}]'
# bbox2_text = f'Size box 2: [{bbox2[3]}, {bbox2[4]},{bbox2[5]}]'
# ax2.text2D(0.5, -0.05, bbox1_text, transform=ax2.transAxes, ha='center', c='black')
# ax2.text2D(0.5, -0.12, bbox2_text, transform=ax2.transAxes, ha='center', c='black')

bbox1 = [0.0, 0.0, 0.0, 2.0, 1.0, 1.0, 0.0]
bbox2 = [0.0, 0.0, 0.0, 2.0, 1.0, 1.0, 0.6]

ax3 = fig.add_subplot(153, projection='3d')
add_plot(ax3, bbox1, bbox2)

# add bbox1 and bbox2 to the plot as text under the plot
# bbox1_text = f'Size box 1: [{bbox1[3]}, {bbox1[4]},{bbox1[5]}]'
# bbox2_text = f'Size box 2: [{bbox2[3]}, {bbox2[4]},{bbox2[5]}]'
# ax3.text2D(0.5, -0.05, bbox1_text, transform=ax3.transAxes, ha='center', c='black')
# ax3.text2D(0.5, -0.12, bbox2_text, transform=ax3.transAxes, ha='center', c='black')

bbox1 = [0.0, 0.0, 0.0, 2.0, 1.0, 1.0, 0.0]
bbox2 = [0.66, 0.33, 0.0, 2.0, 1.0, 1.0, 0.6]

ax4 = fig.add_subplot(154, projection='3d')
add_plot(ax4, bbox1, bbox2)

# add bbox1 and bbox2 to the plot as text under the plot
# bbox1_text = f'Size box 1: [{bbox1[3]}, {bbox1[4]},{bbox1[5]}]'
# bbox2_text = f'Size box 2: [{bbox2[3]}, {bbox2[4]},{bbox2[5]}]'
# ax4.text2D(0.5, -0.05, bbox1_text, transform=ax4.transAxes, ha='center', c='black')
# ax4.text2D(0.5, -0.12, bbox2_text, transform=ax4.transAxes, ha='center', c='black')

bbox1 = [0.0, 0.0, 0.0, 2.0, 1.5, 1.0, 0.0]
bbox2 = [0.33, 0.33, 0.0, 2.0, 1.0, 1.0, 0.0]

ax5 = fig.add_subplot(155, projection='3d')
add_plot(ax5, bbox1, bbox2)

# add bbox1 and bbox2 to the plot as text under the plot
# bbox1_text = f'Size box 1: [{bbox1[3]}, {bbox1[4]},{bbox1[5]}]'
# bbox2_text = f'Size box 2: [{bbox2[3]}, {bbox2[4]},{bbox2[5]}]'
# ax5.text2D(0.5, -0.05, bbox1_text, transform=ax5.transAxes, ha='center', c='black')
# ax5.text2D(0.5, -0.12, bbox2_text, transform=ax5.transAxes, ha='center', c='black')

fig.show()